In [2]:
import os
import librosa
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 Pre Processing Dataset




In [4]:
# Load YAMNet model
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

# Base path: folders like /content/baby_cry_data/BabyCryingSounds/hungry/*.wav
base_dir = "/content/drive/MyDrive/baby_cry_data/Baby Crying Sounds"

# Output base path to save embeddings
output_base = "/content/drive/MyDrive/baby_cry_embeddings"

# Step 1: Helper to load waveform at 16 kHz
def load_audio_16k(file_path):
    waveform, sr = librosa.load(file_path, sr=16000, mono=True)
    return waveform


def augment_audio(waveform, sr=16000):
    choice = random.choice(['pitch', 'stretch', 'none'])

    if choice == 'pitch':
        n_steps = random.choice([-2, -1, 1, 2])
        return librosa.effects.pitch_shift(waveform, sr=sr, n_steps=n_steps)

    elif choice == 'stretch':
        rate = random.uniform(0.8, 1.2)
        return librosa.effects.time_stretch(waveform, rate=rate)

    else:
        return waveform



def extract_yamnet_embedding(file_path, augment=True):
    waveform = load_audio_16k(file_path)

    if augment:
        waveform = augment_audio(waveform, sr=16000)

    waveform = tf.convert_to_tensor(waveform, dtype=tf.float32)
    scores, embeddings, spectrogram = yamnet_model(waveform)
    return tf.reduce_mean(embeddings, axis=0).numpy()



# Step 3: Loop through folders and collect data
X = []
y = []
file_paths = []

labels = sorted([label for label in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, label))])

for label in labels:
    folder = os.path.join(base_dir, label)
    for fname in tqdm(os.listdir(folder), desc=f"Processing {label}"):
        if not fname.lower().endswith(('.wav', '.mp3', '.ogg')):
            continue
        fpath = os.path.join(folder, fname)
        try:
            # Original sample
            emb = extract_yamnet_embedding(fpath, augment=False)
            X.append(emb)
            y.append(label)
            file_paths.append(fpath)

            # Augmented copies (e.g., 2 extra variants)
            for _ in range(2):
                emb_aug = extract_yamnet_embedding(fpath, augment=True)
                X.append(emb_aug)
                y.append(label)
                file_paths.append(fpath)  # or just fpath again
        except Exception as e:
            print(f"⚠️ Error processing {fpath}: {e}")


# Step 4: Encode labels
X = np.array(X)
y = np.array(y)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
label_names = le.classes_

Processing tired: 100%|██████████| 132/132 [00:36<00:00,  3.61it/s]


In [20]:
from collections import Counter

# Count occurrences of each class in the combined dataset
# y_encoded contains the integer encoded labels for all files
class_counts = Counter(y_encoded)

# Map integer indices back to class names
label_counts = {class_names[i]: count for i, count in class_counts.items()}

print("Number of audio files per category:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

Number of audio files per category:
belly pain: 372
burping: 324
cold_hot: 324
discomfort: 405
hungry: 1146
laugh: 324
noise: 324
silence: 324
tired: 396


In [6]:

# Step 5: Split into train, val, test
X_train_val, X_test, y_train_val, y_test, f_train_val, f_test = train_test_split(
    X, y_encoded, file_paths, test_size=0.1, random_state=42, stratify=y_encoded)

X_train, X_val, y_train, y_val, f_train, f_val = train_test_split(
    X_train_val, y_train_val, f_train_val, test_size=0.1, random_state=42, stratify=y_train_val)



In [7]:
splits = {
    'train': (X_train, y_train),
    'val': (X_val, y_val),
    'test': (X_test, y_test)
}

# Step 6: Save to folders by split/class
for split in ['train', 'val', 'test']:
    for class_name in label_names:
        os.makedirs(os.path.join(output_base, split, class_name), exist_ok=True)

def save_embeddings(X_split, y_split, split_name):
    for i, (emb, label_idx) in enumerate(zip(X_split, y_split)):
        class_name = label_names[label_idx]
        save_path = os.path.join(output_base, split_name, class_name, f"yamnet_{i}.npy")
        np.save(save_path, emb)

# Save all splits
save_embeddings(X_train, y_train, "train")
save_embeddings(X_val, y_val, "val")
save_embeddings(X_test, y_test, "test")

print("✅ Done: Embeddings extracted, split, and saved by class.")

✅ Done: Embeddings extracted, split, and saved by class.


In [8]:
import os
import numpy as np

def load_embeddings(data_dir):
    X, y = [], []
    class_names = sorted(os.listdir(data_dir))  # ['belly pain', 'burping', ...]
    label_to_idx = {name: idx for idx, name in enumerate(class_names)}

    for label in class_names:
        label_dir = os.path.join(data_dir, label)
        for fname in os.listdir(label_dir):
            if fname.endswith('.npy'):
                path = os.path.join(label_dir, fname)
                emb = np.load(path)
                X.append(emb)
                y.append(label_to_idx[label])
    return np.array(X), np.array(y), class_names


In [9]:
train_dir = '/content/drive/MyDrive/baby_cry_embeddings/train'
val_dir   = '/content/drive/MyDrive/baby_cry_embeddings/val'
test_dir  = '/content/drive/MyDrive/baby_cry_embeddings/test'

X_train, y_train, class_names = load_embeddings(train_dir)
X_val, y_val, _ = load_embeddings(val_dir)
X_test, y_test, _ = load_embeddings(test_dir)

print(f"✅ Loaded: {X_train.shape[0]} train | {X_val.shape[0]} val | {X_test.shape[0]} test")


✅ Loaded: 5940 train | 1103 val | 1316 test


In [16]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers

# Build the model with best hyperparameters
model = tf.keras.Sequential([
    layers.Input(shape=(1024,)),

    # Layer 1
    layers.Dense(526, use_bias=False,
                 kernel_regularizer=regularizers.l2(0.0001897)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.4),

    # Layer 2
    layers.Dense(256, use_bias=False,
                 kernel_regularizer=regularizers.l2(0.0002384)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.4),


    layers.Dense(128, use_bias=False,
    kernel_regularizer=regularizers.l2(0.0002384)),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    # Output layer
    layers.Dense(len(np.unique(y_train)), activation="softmax")
])

# Compile
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001342)

model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

early_stop = tf.keras.callbacks.EarlyStopping(
    patience=15, restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.5, patience=7, verbose=1, min_lr=1e-6
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "best_yamnet_model.h5", save_best_only=True, verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr, checkpoint],
    # class_weight=class_weight_dict
)


Epoch 1/100
183/186 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2839 - loss: 2.2817
Epoch 1: val_loss improved from inf to 1.92460, saving model to best_yamnet_model.h5


186/186 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.2856 - loss: 2.2781 - val_accuracy: 0.5014 - val_loss: 1.9246 - learning_rate: 1.3420e-04
Epoch 2/100
183/186 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4742 - loss: 1.8264
Epoch 2: val_loss improved from 1.92460 to 1.62403, saving model to best_yamnet_model.h5


186/186 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.4744 - loss: 1.8254 - val_accuracy: 0.5530 - val_loss: 1.6240 - learning_rate: 1.3420e-04
Epoch 3/100
186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4992 - loss: 1.6923
Epoch 3: val_loss improved from 1.62403 to 1.52356, saving model to best_yamnet_model.h5


186/186 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4992 - loss: 1.6922 - val_accuracy: 0.5603 - val_loss: 1.5236 - learning_rate: 1.3420e-04
Epoch 4/100
185/186 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5191 - loss: 1.5961
Epoch 4: val_loss improved from 1.52356 to 1.46584, saving model to best_yamnet_model.h5


186/186 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5192 - loss: 1.5960 - val_accuracy: 0.5657 - val_loss: 1.4658 - learning_rate: 1.3420e-04
Epoch 5/100
186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5215 - loss: 1.5472
Epoch 5: val_loss improved from 1.46584 to 1.43895, saving model to best_yamnet_model.h5


...

186/186 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.6906 - loss: 0.8766 - val_accuracy: 0.6156 - val_loss: 0.9251 - learning_rate: 1.3420e-04
Epoch 83/100
184/186 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7014 - loss: 0.8565
Epoch 83: val_loss improved from 0.92510 to 0.92443, saving model to best_yamnet_model.h5


186/186 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7013 - loss: 0.8567 - val_accuracy: 0.6210 - val_loss: 0.9244 - learning_rate: 1.3420e-04
Epoch 84/100
183/186 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6929 - loss: 0.8754
Epoch 84: val_loss did not improve from 0.92443
186/186 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6928 - loss: 0.8755 - val_accuracy: 0.6047 - val_loss: 0.9499 - learning_rate: 1.3420e-04
Epoch 85/100
184/186 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6814 - loss: 0.8687
Epoch 85: val_loss did not improve from 0.92443
186/186 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.6815 - loss: 0.8688 - val_accuracy: 0.6210 - val_loss: 0.9246 - learning_rate: 1.3420e-04
Epoch 86/100
183/186 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6856 - loss: 0.8525
Epoch 86: val_loss did not improve from 0.92443
186/186 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6856 - loss: 0.8527 - val_accuracy: 0.6201 - val_loss: 0.9279 - learning_rate: 1.3420e-04
Epoch

186/186 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6987 - loss: 0.8451 - val_accuracy: 0.6102 - val_loss: 0.9173 - learning_rate: 6.7100e-05
Epoch 92/100
186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7155 - loss: 0.8136
Epoch 92: val_loss improved from 0.91728 to 0.90123, saving model to best_yamnet_model.h5


186/186 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7154 - loss: 0.8136 - val_accuracy: 0.6156 - val_loss: 0.9012 - learning_rate: 6.7100e-05
Epoch 93/100
185/186 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7237 - loss: 0.7951
Epoch 93: val_loss did not improve from 0.90123
186/186 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7236 - loss: 0.7952 - val_accuracy: 0.6138 - val_loss: 0.9150 - learning_rate: 6.7100e-05
Epoch 94/100
186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7031 - loss: 0.8120
Epoch 94: val_loss did not improve from 0.90123
186/186 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.7031 - loss: 0.8120 - val_accuracy: 0.6147 - val_loss: 0.9096 - learning_rate: 6.7100e-05
Epoch 95/100
183/186 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7124 - loss: 0.8020
Epoch 95: val_loss did not improve from 0.90123
186/186 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7123 - loss: 0.8022 - val_accuracy: 0.6111 - val_loss: 0.9098 - learning_rate: 6.7100e-05
Epoch

186/186 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7302 - loss: 0.7723 - val_accuracy: 0.6292 - val_loss: 0.9012 - learning_rate: 3.3550e-05


In [13]:
from sklearn.metrics import classification_report, accuracy_score
# Get predictions as class indices
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\n✅ Final Test Accuracy: {test_acc:.2%}")

y_pred = model.predict(X_test).argmax(axis=1)

print(classification_report(y_test, y_pred, target_names=class_names))


42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4787 - loss: 1.1388

✅ Final Test Accuracy: 56.53%
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
              precision    recall  f1-score   support

  belly pain       0.61      0.57      0.59       129
     burping       0.58      0.44      0.50       112
    cold_hot       0.37      0.17      0.23       112
  discomfort       0.45      0.26      0.33       137
      hungry       0.34      0.51      0.41       354
       laugh       1.00      1.00      1.00       113
       noise       0.99      1.00      1.00       113
     silence       1.00      1.00      1.00       113
       tired       0.42      0.35      0.38       133

    accuracy                           0.57      1316
   macro avg       0.64      0.59      0.60      1316
weighted avg       0.58      0.57      0.56      1316



In [14]:
!pip install keras_tuner
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras import layers, regularizers

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(1024,)))

    for i in range(hp.Int("num_layers", 1, 3)):
        units = hp.Int(f"units_{i}", min_value=64, max_value=512, step=64)
        l2 = hp.Float(f"l2_{i}", 1e-4, 1e-2, sampling="log")

        model.add(layers.Dense(units, use_bias=False,
                               kernel_regularizer=regularizers.l2(l2)))
        model.add(layers.BatchNormalization())
        model.add(layers.Activation('relu'))

        dropout = hp.Float(f"dropout_{i}", 0.3, 0.6, step=0.1)
        model.add(layers.Dropout(dropout))

    model.add(layers.Dense(len(np.unique(y_train)), activation="softmax"))

    lr = hp.Float("lr", 1e-4, 1e-2, sampling="log")
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model


from keras_tuner.tuners import BayesianOptimization

tuner = BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=20,
    directory="yamnet_tuning",
    project_name="bayesian_final"
)

early_stop = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
tuner.search(X_train, y_train, epochs=30, validation_data=(X_val, y_val), callbacks=[early_stop, checkpoint])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.2 MB/s eta 0:00:00


KeyboardInterrupt: 

In [11]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(1)[0]

print("Best hyperparameters:")
print(best_hp.values)

# Optional: Evaluate
loss, acc = best_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Best hyperparameters:
{'num_layers': 2, 'units_0': 128, 'l2_0': 0.00018974491572285046, 'dropout_0': 0.4, 'lr': 0.00013423673657073052, 'units_1': 512, 'l2_1': 0.00023844032090936112, 'dropout_1': 0.4, 'units_2': 320, 'l2_2': 0.003269038958697772, 'dropout_2': 0.3}
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4824 - loss: 1.2745
Test Accuracy: 0.58


In [18]:
from IPython.display import Audio

true_labels = [class_names[i] for i in y_test]
pred_labels = [class_names[i] for i in y_pred]

for i in range(10):  # show first 10
    print(f"File: {f_test[i]}")
    print(f"True label: {true_labels[i]}")
    print(f"Predicted:  {pred_labels[i]}")
    print("-" * 40)


i = 212  # index of interest
print(f"True: {true_labels[i]}, Predicted: {pred_labels[i]}")
Audio(filename=f_test[i])  # Use filename for file paths

File: /content/drive/MyDrive/baby_cry_data/Baby Crying Sounds/hungry/8f5c01dc-6826-4076-a831-027ad61afdaf-1435873509176-1.7-f-26-hu.wav
True label: belly pain
Predicted:  hungry
----------------------------------------
File: /content/drive/MyDrive/baby_cry_data/Baby Crying Sounds/belly pain/431c.wav
True label: belly pain
Predicted:  hungry
----------------------------------------
File: /content/drive/MyDrive/baby_cry_data/Baby Crying Sounds/burping/483c.wav
True label: belly pain
Predicted:  hungry
----------------------------------------
File: /content/drive/MyDrive/baby_cry_data/Baby Crying Sounds/hungry/a56e2494-d2b2-480e-af0b-de9ea8f33572-1436897427825-1.7-f-26-hu.wav
True label: belly pain
Predicted:  belly pain
----------------------------------------
File: /content/drive/MyDrive/baby_cry_data/Baby Crying Sounds/hungry/8c66887c-8251-49fa-903b-c02d56405f58-1436856137837-1.7-m-26-hu.wav
True label: belly pain
Predicted:  hungry
----------------------------------------
File: /conte

Demo

In [19]:
from IPython.display import display, HTML
import base64
import io

display(HTML('''
<style>
    .colab-container {
        font-family: 'Google Sans', -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
        background: linear-gradient(135deg, #4285f4 0%, #34a853 50%, #ea4335 100%);
        margin: -8px -8px 16px -8px;
        border-radius: 8px;
        overflow: hidden;
        box-shadow: 0 4px 12px rgba(0,0,0,0.15);
        max-width: 100%;
    }

    .app-header {
        background: rgba(255, 255, 255, 0.95);
        backdrop-filter: blur(10px);
        padding: 24px;
        border-bottom: 1px solid rgba(0,0,0,0.1);
    }

    .header-content {
        display: flex;
        align-items: center;
        gap: 16px;
    }

    .app-icon {
        width: 48px;
        height: 48px;
        background: linear-gradient(135deg, #4285f4, #34a853);
        border-radius: 12px;
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 20px;
        color: white;
        flex-shrink: 0;
    }

    .header-text h1 {
        font-size: 24px;
        font-weight: 500;
        color: #1a73e8;
        margin: 0 0 4px 0;
        letter-spacing: -0.2px;
    }

    .header-text p {
        font-size: 14px;
        color: #5f6368;
        margin: 0;
        font-weight: 400;
    }

    .main-content {
        background: white;
        padding: 32px 24px;
    }

    .upload-zone {
        border: 2px dashed #dadce0;
        border-radius: 12px;
        padding: 40px 20px;
        text-align: center;
        transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
        background: #fafbfc;
        position: relative;
        cursor: pointer;
    }

    .upload-zone:hover {
        border-color: #4285f4;
        background: rgba(66, 133, 244, 0.04);
    }

    .upload-zone.dragover {
        border-color: #4285f4;
        background: rgba(66, 133, 244, 0.08);
        transform: scale(1.02);
    }

    .upload-icon-container {
        width: 64px;
        height: 64px;
        margin: 0 auto 16px;
        background: linear-gradient(135deg, #4285f4, #34a853);
        border-radius: 16px;
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 24px;
        color: white;
        box-shadow: 0 2px 8px rgba(66, 133, 244, 0.3);
    }

    .upload-title {
        font-size: 18px;
        font-weight: 500;
        color: #202124;
        margin-bottom: 8px;
    }

    .upload-subtitle {
        font-size: 14px;
        color: #5f6368;
        margin-bottom: 24px;
        line-height: 1.4;
    }

    .file-input {
        display: none;
    }

    .file-button {
        display: inline-flex;
        align-items: center;
        gap: 8px;
        background: #4285f4;
        color: white;
        border: none;
        padding: 12px 24px;
        border-radius: 24px;
        font-size: 14px;
        font-weight: 500;
        cursor: pointer;
        transition: all 0.2s cubic-bezier(0.4, 0, 0.2, 1);
        text-transform: none;
        letter-spacing: 0.25px;
    }

    .file-button:hover {
        background: #3367d6;
        box-shadow: 0 2px 8px rgba(66, 133, 244, 0.4);
        transform: translateY(-1px);
    }

    .file-info {
        margin-top: 20px;
        padding: 16px;
        background: #e8f0fe;
        border: 1px solid #d2e3fc;
        border-radius: 8px;
        display: none;
    }

    .file-details {
        display: flex;
        align-items: center;
        gap: 12px;
    }

    .file-icon {
        width: 32px;
        height: 32px;
        background: #4285f4;
        border-radius: 8px;
        display: flex;
        align-items: center;
        justify-content: center;
        color: white;
        font-size: 14px;
        flex-shrink: 0;
    }

    .file-text {
        flex: 1;
    }

    .file-name {
        font-size: 14px;
        font-weight: 500;
        color: #1a73e8;
        margin-bottom: 2px;
    }

    .file-size {
        font-size: 12px;
        color: #5f6368;
    }

    .action-section {
        margin-top: 32px;
        text-align: center;
    }

    .analyze-button {
        background: linear-gradient(135deg, #34a853, #4285f4);
        color: white;
        border: none;
        padding: 16px 32px;
        border-radius: 28px;
        font-size: 16px;
        font-weight: 500;
        cursor: pointer;
        transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
        min-width: 160px;
        position: relative;
        overflow: hidden;
        letter-spacing: 0.25px;
    }

    .analyze-button:hover:not(:disabled) {
        transform: translateY(-2px);
        box-shadow: 0 6px 20px rgba(52, 168, 83, 0.4);
    }

    .analyze-button:disabled {
        opacity: 0.6;
        cursor: not-allowed;
        transform: none;
    }

    .analyze-button.loading {
        color: transparent;
    }

    .analyze-button.loading::after {
        content: '';
        position: absolute;
        top: 50%;
        left: 50%;
        width: 20px;
        height: 20px;
        margin: -10px 0 0 -10px;
        border: 2px solid rgba(255,255,255,0.3);
        border-top: 2px solid white;
        border-radius: 50%;
        animation: spin 1s linear infinite;
    }

    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }

    .result-container {
        margin-top: 24px;
        opacity: 0;
        transform: translateY(20px);
        transition: all 0.4s cubic-bezier(0.4, 0, 0.2, 1);
    }

    .result-container.show {
        opacity: 1;
        transform: translateY(0);
    }

    .result-card {
        padding: 24px;
        border-radius: 12px;
        text-align: center;
        position: relative;
        overflow: hidden;
    }

    .result-success {
        background: linear-gradient(135deg, #34a853, #4285f4);
        color: white;
        box-shadow: 0 4px 12px rgba(52, 168, 83, 0.3);
    }

    .result-error {
        background: linear-gradient(135deg, #ea4335, #fbbc04);
        color: white;
        box-shadow: 0 4px 12px rgba(234, 67, 53, 0.3);
    }

    .result-warning {
        background: linear-gradient(135deg, #fbbc04, #ff6d01);
        color: white;
        box-shadow: 0 4px 12px rgba(251, 188, 4, 0.3);
    }

    .result-icon {
        font-size: 32px;
        margin-bottom: 12px;
        display: block;
    }

    .result-title {
        font-size: 18px;
        font-weight: 500;
        margin-bottom: 8px;
        letter-spacing: 0.1px;
    }

    .result-text {
        font-size: 14px;
        opacity: 0.9;
        line-height: 1.4;
    }

    .confidence-bar {
        width: 100%;
        height: 4px;
        background: rgba(255,255,255,0.3);
        border-radius: 2px;
        margin-top: 16px;
        overflow: hidden;
    }

    .confidence-fill {
        height: 100%;
        background: rgba(255,255,255,0.8);
        border-radius: 2px;
        transition: width 1s cubic-bezier(0.4, 0, 0.2, 1);
        width: 0%;
    }

    .footer-section {
        background: #f8f9fa;
        padding: 20px 24px;
        border-top: 1px solid #e8eaed;
        text-align: center;
    }

    .tech-info {
        display: flex;
        align-items: center;
        justify-content: center;
        gap: 16px;
        flex-wrap: wrap;
    }

    .tech-badge {
        display: inline-flex;
        align-items: center;
        gap: 6px;
        background: #4285f4;
        color: white;
        padding: 6px 12px;
        border-radius: 16px;
        font-size: 11px;
        font-weight: 500;
        text-transform: uppercase;
        letter-spacing: 0.5px;
    }

    .status-indicator {
        width: 8px;
        height: 8px;
        background: #34a853;
        border-radius: 50%;
        animation: pulse 2s infinite;
    }

    @keyframes pulse {
        0% { opacity: 1; }
        50% { opacity: 0.5; }
        100% { opacity: 1; }
    }

    .processing-steps {
        display: none;
        margin-top: 20px;
        text-align: left;
    }

    .processing-steps.show {
        display: block;
    }

    .step {
        display: flex;
        align-items: center;
        gap: 12px;
        padding: 8px 0;
        font-size: 13px;
        color: #5f6368;
    }

    .step-icon {
        width: 16px;
        height: 16px;
        border-radius: 50%;
        background: #e8eaed;
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 10px;
        flex-shrink: 0;
    }

    .step.active .step-icon {
        background: #4285f4;
        color: white;
    }

    .step.completed .step-icon {
        background: #34a853;
        color: white;
    }

    @media (max-width: 600px) {
        .colab-container {
            margin: -8px -16px 16px -16px;
        }

        .app-header, .main-content, .footer-section {
            padding-left: 16px;
            padding-right: 16px;
        }

        .header-content {
            flex-direction: column;
            text-align: center;
            gap: 12px;
        }

        .upload-zone {
            padding: 32px 16px;
        }

        .tech-info {
            flex-direction: column;
            gap: 8px;
        }
    }
</style>

<div class="colab-container">
    <div class="app-header">
        <div class="header-content">
            <div class="app-icon">🍼</div>
            <div class="header-text">
                <h1>Baby Crying Classifier</h1>
                <p>Advanced AI-powered infant behavior analysis system</p>
            </div>
        </div>
    </div>

    <div class="main-content">
        <div class="upload-zone" id="uploadZone" onclick="document.getElementById('audioFile').click()">
            <div class="upload-icon-container">
                <span>🎵</span>
            </div>
            <div class="upload-title">Upload Audio Sample</div>
            <div class="upload-subtitle">
                Drop your audio file here or click to browse<br>
                <strong>Supported:</strong> WAV, MP3, M4A, AAC • <strong>Max:</strong> 25MB
            </div>
            <button class="file-button" type="button">
                <span>📁</span> Choose File
            </button>
            <input type="file" id="audioFile" class="file-input" accept=".wav,.mp3,.m4a,.aac">
        </div>

        <div id="fileInfo" class="file-info">
            <div class="file-details">
                <div class="file-icon">🎵</div>
                <div class="file-text">
                    <div class="file-name" id="fileName"></div>
                    <div class="file-size" id="fileSize"></div>
                </div>
            </div>
        </div>

        <div class="processing-steps" id="processingSteps">
            <div class="step" id="step1">
                <div class="step-icon">1</div>
                <span>Loading audio file...</span>
            </div>
            <div class="step" id="step2">
                <div class="step-icon">2</div>
                <span>Extracting acoustic features...</span>
            </div>
            <div class="step" id="step3">
                <div class="step-icon">3</div>
                <span>Neural network analysis...</span>
            </div>
            <div class="step" id="step4">
                <div class="step-icon">4</div>
                <span>Generating classification...</span>
            </div>
        </div>

        <div class="action-section">
            <button class="analyze-button" id="analyzeBtn" onclick="analyzeCrying()" disabled>
                🔬 Analyze Audio
            </button>
        </div>

        <div id="resultContainer" class="result-container">
            <div id="resultCard" class="result-card">
                <span class="result-icon" id="resultIcon"></span>
                <div class="result-title" id="resultTitle"></div>
                <div class="result-text" id="resultText"></div>
                <div class="confidence-bar">
                    <div class="confidence-fill" id="confidenceFill"></div>
                </div>
            </div>
        </div>
    </div>

    <div class="footer-section">
        <div class="tech-info">
            <div class="tech-badge">
                <div class="status-indicator"></div>
                ML Ready
            </div>
            <div class="tech-badge">
                <span>⚡</span> Real-time
            </div>
            <div class="tech-badge">
                <span>🔒</span> Secure
            </div>
        </div>
    </div>
</div>

<script>
    // Global variables
    let selectedFile = null;
    const fileInput = document.getElementById('audioFile');
    const uploadZone = document.getElementById('uploadZone');
    const fileInfo = document.getElementById('fileInfo');
    const analyzeBtn = document.getElementById('analyzeBtn');
    const resultContainer = document.getElementById('resultContainer');
    const processingSteps = document.getElementById('processingSteps');

    // File handling
    fileInput.addEventListener('change', handleFileSelection);

    // Drag and drop
    uploadZone.addEventListener('dragover', handleDragOver);
    uploadZone.addEventListener('dragleave', handleDragLeave);
    uploadZone.addEventListener('drop', handleDrop);

    function handleDragOver(e) {
        e.preventDefault();
        uploadZone.classList.add('dragover');
    }

    function handleDragLeave(e) {
        e.preventDefault();
        uploadZone.classList.remove('dragover');
    }

    function handleDrop(e) {
        e.preventDefault();
        uploadZone.classList.remove('dragover');
        const files = e.dataTransfer.files;
        if (files.length > 0) {
            processFile(files[0]);
        }
    }

    function handleFileSelection(e) {
        if (e.target.files.length > 0) {
            processFile(e.target.files[0]);
        }
    }

    function processFile(file) {
        // Validate file
        if (!isValidAudio(file)) {
            showResult('❌', 'Invalid File Type',
                'Please select a valid audio file (WAV, MP3, M4A, AAC)', 'error');
            return;
        }

        if (file.size > 25 * 1024 * 1024) {
            showResult('⚠️', 'File Too Large',
                'Please select a file smaller than 25MB', 'warning');
            return;
        }

        selectedFile = file;

        // Update UI
        document.getElementById('fileName').textContent = file.name;
        document.getElementById('fileSize').textContent = formatFileSize(file.size);
        fileInfo.style.display = 'block';
        analyzeBtn.disabled = false;

        // Hide previous results
        resultContainer.classList.remove('show');
    }

    function isValidAudio(file) {
        const validTypes = ['audio/wav', 'audio/mpeg', 'audio/mp3', 'audio/mp4', 'audio/m4a', 'audio/aac'];
        const validExts = ['.wav', '.mp3', '.m4a', '.aac'];

        return validTypes.includes(file.type) ||
               validExts.some(ext => file.name.toLowerCase().endsWith(ext));
    }

    function formatFileSize(bytes) {
        if (bytes === 0) return '0 Bytes';
        const k = 1024;
        const sizes = ['Bytes', 'KB', 'MB', 'GB'];
        const i = Math.floor(Math.log(bytes) / Math.log(k));
        return parseFloat((bytes / Math.pow(k, i)).toFixed(2)) + ' ' + sizes[i];
    }

    async function analyzeCrying() {
        if (!selectedFile) {
            showResult('⚠️', 'No File Selected',
                'Please select an audio file first', 'warning');
            return;
        }

        // Start analysis
        analyzeBtn.classList.add('loading');
        analyzeBtn.disabled = true;
        processingSteps.classList.add('show');
        resultContainer.classList.remove('show');

        // Simulate processing steps
        await simulateProcessing();

        try {
            // Create form data
            const formData = new FormData();
            formData.append('file', selectedFile);

            // Make API call (replace with your endpoint)
            const response = await fetch('/api/classify-crying', {
                method: 'POST',
                body: formData
            });

            if (response.ok) {
                const result = await response.json();
                showResult('✅', 'Analysis Complete',
                    `Classification: ${result.label}`, 'success', result.confidence);
            } else {
                throw new Error('API Error');
            }
        } catch (error) {
            // Demo mode with realistic results
            const demoResults = [
                { label: 'Hunger Cry', confidence: 0.89, icon: '🍼' },
                { label: 'Discomfort', confidence: 0.76, icon: '😣' },
                { label: 'Sleepy/Tired', confidence: 0.84, icon: '😴' },
                { label: 'Pain/Distress', confidence: 0.71, icon: '😢' },
                { label: 'Attention Seeking', confidence: 0.82, icon: '👶' }
            ];

            const randomResult = demoResults[Math.floor(Math.random() * demoResults.length)];

            setTimeout(() => {
                showResult(randomResult.icon, 'Classification Result',
                    `Detected: ${randomResult.label}`, 'success', randomResult.confidence);
            }, 500);
        } finally {
            // Reset UI
            setTimeout(() => {
                analyzeBtn.classList.remove('loading');
                analyzeBtn.disabled = false;
                processingSteps.classList.remove('show');
                resetProcessingSteps();
            }, 2000);
        }
    }

    async function simulateProcessing() {
        const steps = ['step1', 'step2', 'step3', 'step4'];

        for (let i = 0; i < steps.length; i++) {
            await new Promise(resolve => setTimeout(resolve, 400));
            const step = document.getElementById(steps[i]);
            step.classList.add('active');

            await new Promise(resolve => setTimeout(resolve, 300));
            step.classList.remove('active');
            step.classList.add('completed');
        }
    }

    function resetProcessingSteps() {
        const steps = ['step1', 'step2', 'step3', 'step4'];
        steps.forEach(stepId => {
            const step = document.getElementById(stepId);
            step.classList.remove('active', 'completed');
        });
    }

    function showResult(icon, title, text, type, confidence = null) {
        const resultCard = document.getElementById('resultCard');
        const resultIcon = document.getElementById('resultIcon');
        const resultTitle = document.getElementById('resultTitle');
        const resultText = document.getElementById('resultText');
        const confidenceFill = document.getElementById('confidenceFill');

        resultIcon.textContent = icon;
        resultTitle.textContent = title;
        resultText.textContent = text;

        resultCard.className = `result-card result-${type}`;

        if (confidence) {
            confidenceFill.style.width = `${confidence * 100}%`;
            resultText.textContent += ` (${(confidence * 100).toFixed(1)}% confidence)`;
        } else {
            confidenceFill.style.width = '0%';
        }

        setTimeout(() => {
            resultContainer.classList.add('show');
        }, 100);
    }

    // Initialize
    document.addEventListener('DOMContentLoaded', function() {
        // Reset form
        fileInput.value = '';
        fileInfo.style.display = 'none';
        analyzeBtn.disabled = true;
        processingSteps.classList.remove('show');
        resultContainer.classList.remove('show');
    });
</script>
'''))